In [1]:
pip install pyngrok

In [67]:
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pickle
from io import BytesIO
import traceback
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import tensorflow
import glob
import cv2
from skimage.metrics import structural_similarity as ssim

In [68]:
import zipfile
with zipfile.ZipFile('/content/template.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/template')

In [69]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
last_conv_layer = base_model.get_layer('block5_pool').output
x = GlobalAveragePooling2D()(last_conv_layer)
x = Dense(4096, activation='relu')(x)  # Add a Dense layer with 4096 units
feature_extractor = tensorflow.keras.Model(inputs=base_model.input, outputs=x)

In [70]:
model = load_model('/content/VGG16Epoch20poch.keras')

In [71]:
with open('/content/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)
max_length = 34

In [72]:
with open('/content/features.pkl', 'rb') as f:
    features = pickle.load(f)

In [73]:
app = Flask(__name__)

In [74]:
NGROK_AUTH_TOKEN = "2pzJ3n8N6o3RVZ8fPxEtEgbkeFA_81N1eBZsukBB2cJmUrGu"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000)
print(f"Ngrok Tunnel URL: {public_url}")

Ngrok Tunnel URL: NgrokTunnel: "https://2b98-34-148-70-236.ngrok-free.app" -> "http://localhost:5000"


In [75]:
def load_templates(template_folder):
    templates = []
    for file_path in glob.glob(template_folder + "/*.png"):
        template = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        template = cv2.resize(template, (256, 256))
        templates.append(template)
    return templates

In [76]:
def preprocess_image_for_template_matching(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (256, 256))
    return image

In [77]:
def is_xray_image(image, templates, threshold=0.5):
    for template in templates:
        score, _ = ssim(image, template, full=True)
        if score >= threshold:
            return True  # Match found
    return False

In [78]:
def validate_xray(image_path, template_folder):
    templates = load_templates(template_folder)
    image = preprocess_image_for_template_matching(image_path)
    if is_xray_image(image, templates):
        return "Valid X-ray image"
    else:
        return "Invalid image. Not an X-ray."

In [79]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [80]:
def preprocess_image_for_captioning(image_path):
  img = load_img(image_path, target_size=(224, 224))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  features = feature_extractor.predict(img)
  return features

In [81]:
def predict_caption(model, image, tokenizer, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=150, padding='post')

        # Assuming your model handles both image and sequence input
        yhat = model.predict([image, sequence], verbose=0)

        yhat = np.argmax(yhat)
        word = idx_to_word(yhat, tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == 'endseq':
            break
    return in_text

In [82]:
def preprocess_image(file_path):  # Changed parameter to file_path
    img = load_img(file_path, target_size=(224, 224))  # Use file_path here
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = feature_extractor.predict(img)
    return features

In [83]:
@app.route('/upload', methods=['POST'])
def upload():
    try:
        if 'file' not in request.files:
            raise ValueError('No file uploaded.')

        file = request.files['file']
        if file:
            # 1. Save the uploaded file temporarily:
            temp_file_path = os.path.join('/content', file.filename)  # Create a temporary file path
            file.save(temp_file_path)  # Save the file

            # 2. Validate the image using template matching:
            validation_result = validate_xray(temp_file_path, '/content/template')  # Use temp_file_path
            if validation_result == "Valid X-ray image":
                # 3. Preprocess image using the saved file path:
                features = preprocess_image(temp_file_path)  # Use temp_file_path

                # Generate caption using your model and extracted features
                caption = predict_caption(model, features, tokenizer, max_length)

                return jsonify({'report': caption.replace('startseq', '').replace('endseq', '').strip()})
            else:
                return jsonify({'error': validation_result})  # Return error if not an X-ray

        else:
            raise ValueError('File processing error.')

    except Exception as e:
        print(f"Error: {str(e)}")
        print(traceback.format_exc())
        return jsonify({'error': f'Error generating report: {str(e)}'})

In [84]:
@app.route('/')
def index():
    return render_template_string('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>X-Ray Report Generator</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #121212;
            color: #e0e0e0;
        }
        header {
            background: #1f1f1f;
            color: #76c7c0;
            padding: 1rem 0;
            text-align: center;
        }
        main {
            max-width: 800px;
            margin: 2rem auto;
            padding: 1rem;
            background: #1f1f1f;
            border-radius: 8px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.8);
        }
        h1 {
            font-size: 2rem;
            margin-bottom: 1rem;
        }
        .upload-section {
            text-align: center;
            margin-bottom: 2rem;
        }
        .upload-section input[type="file"] {
            margin-top: 1rem;
            padding: 0.5rem;
            border-radius: 5px;
            border: 1px solid #555;
            background-color: #2c2c2c;
            color: #e0e0e0;
            cursor: pointer;
        }
        .image-preview {
            margin: 2rem 0;
            text-align: center;
        }
        .image-preview img {
            max-width: 100%;
            height: auto;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.8);
        }
        .report {
            margin-top: 2rem;
            text-align: center;
        }
        .report textarea {
            width: 100%;
            height: 150px;
            padding: 1rem;
            font-size: 1rem;
            border: 1px solid #555;
            border-radius: 8px;
            background-color: #2c2c2c;
            color: #e0e0e0;
        }
        .report button {
            margin-top: 1rem;
            padding: 0.8rem 2rem;
            background-color: #76c7c0;
            border: none;
            border-radius: 5px;
            color: #121212;
            font-size: 1rem;
            cursor: pointer;
        }
        .report button:hover {
            background-color: #64b0a4;
        }
    </style>
</head>
<body>
    <header>
        <h1>X-Ray Report Generator</h1>
    </header>
    <main>
        <section>
            <h2 style="text-align: center; margin-bottom: 1rem;">X-Ray Image Analysis</h2>
        </section>
        <section class="upload-section">
            <h2>Upload X-Ray Image</h2>
            <input type="file" id="fileInput" accept="image/*" onchange="handleFileUpload()">
        </section>
        <section class="image-preview" id="imagePreview">
            <h3>Uploaded Image Preview</h3>
            <p>No image uploaded yet.</p>
        </section>
        <section class="report">
            <h3>Generated Report</h3>
            <textarea id="report" placeholder="The report will appear here after analysis..."></textarea>
            <button onclick="generateReport()">Generate Report</button>
        </section>
    </main>
    <script>
        let uploadedFile = null;

        function handleFileUpload() {
            const fileInput = document.getElementById('fileInput');
            const imagePreview = document.getElementById('imagePreview');
            const report = document.getElementById('report');

            uploadedFile = fileInput.files[0];

            if (uploadedFile) {
                const reader = new FileReader();
                reader.onload = function (e) {
                    imagePreview.innerHTML = `<img src="${e.target.result}" alt="Uploaded X-Ray Image">`;
                    report.value = ""; // Clear the report text area
                };
                reader.readAsDataURL(uploadedFile);
            } else {
                imagePreview.innerHTML = '<p>No image uploaded yet.</p>';
                report.value = '';
            }
        }

        async function generateReport() {
            const report = document.getElementById('report');
            if (!uploadedFile) {
                report.value = "Please upload an X-Ray image first.";
                return;
            }

            const formData = new FormData();
            formData.append('file', uploadedFile);

            try {
                const response = await fetch('/upload', {
                    method: 'POST',
                    body: formData
                });

                const data = await response.json();
                if (data.report) {
                    report.value = data.report;
                } else {
                    report.value = "Error generating report.";
                }
            } catch (error) {
                console.error(error);
                report.value = "Error generating report.";
            }
        }
    </script>
</body>
</html>
    ''')

In [85]:
if __name__ == "__main__":
    print(f"Ngrok Tunnel URL: {public_url}")
    app.run(port=5000)

Ngrok Tunnel URL: NgrokTunnel: "https://2b98-34-148-70-236.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:16:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:16:16] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 864ms/step


INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:16:35] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:16:48] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:17:03] "POST /upload HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step


INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:17:21] "POST /upload HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step


INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:17:35] "POST /upload HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step


INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:17:53] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:18:09] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:18:23] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2024 16:20:42] "POST /upload HTTP/1.1" 200 -
